<!-- 9) Viết chương trình đổi màu HSV ngẫu nhiên nhưng không trùng của các hình trong thư mục Exercise. -->

In [ ]:
import numpy as np
import imageio.v2 as iio
import scipy.ndimage as sn
import os
import glob
import colorsys

input_folder = 'Exercise'
output_folder = 'Exercise_RandomHSV'
os.makedirs(output_folder, exist_ok=True)

image_paths = glob.glob(os.path.join(input_folder, '*'))

# Tạo danh sách các hoán vị không trùng nhau cho số ảnh
from itertools import permutations
perm_list = list(permutations([0, 1, 2]))
np.random.shuffle(perm_list)
assert len(perm_list) >= len(image_paths), "Số ảnh vượt quá số hoán vị kênh HSV!"

for idx, img_path in enumerate(image_paths):
    img = iio.imread(img_path)
    
    if img.ndim == 3:
        denoised = np.zeros_like(img)
        for c in range(3):
            denoised[:, :, c] = sn.gaussian_filter(img[:, :, c], sigma=1)
    else:
        denoised = sn.gaussian_filter(img, sigma=1)
    
    if denoised.ndim == 3 and denoised.shape[2] == 3:

        img_norm = denoised / 255.0
        hsv_img = np.zeros_like(img_norm)
        for i in range(img_norm.shape[0]):
            for j in range(img_norm.shape[1]):
                hsv_img[i, j] = colorsys.rgb_to_hsv(*img_norm[i, j])

        perm = perm_list[idx]
        hsv_permuted = hsv_img[:, :, perm]

        rgb_new = np.zeros_like(hsv_permuted)
        for i in range(hsv_permuted.shape[0]):
            for j in range(hsv_permuted.shape[1]):
                rgb_new[i, j] = colorsys.hsv_to_rgb(*hsv_permuted[i, j])
        rgb_new = (rgb_new * 255).astype(np.uint8)
    else:
        rgb_new = denoised
    
    base_name = os.path.basename(img_path)
    save_path = os.path.join(output_folder, f'randomHSV_{base_name}')
    iio.imwrite(save_path, rgb_new)

print("Đã xử lý xong tất cả ảnh.")  